In [ ]:
"""
I had a idea from work like below for my final projects, I think I will doing solo but if anyone want to work together you are more than welcome to DM me. 

Background: My job is sale directors for steel companies, so we hitting the road really often, when we going out to visit customer, we will visit 3-4 customer at the same time. Everytime I need to find information like what is the sale volume to customer last month, what is the sizes, what is the average prices or previous month prices, I need to open my laptop, get to the ERP and found the information. 

My project: I will made use a local LLMs and Perplexity as my personal assistance. So I will make RAG to access the database, take out the information I need, example like the list of sizes we sold to customers. The material description can be change but can be also repeating multiple times, the diffirences is in the volume wise.


Interface: I think I will use Gradio to do chat interface, then deploy to Hughing Face with address that only me can access 

Extra Credit: It is already a structured data to begin with hahaha. I am thinking about routing agents as tools call.
There will be 3 tools calling:
1/Use PandasQueryEngines to get information of averages prices etc...
2/Use Tools to drawing a charts using data from PandasQuery Engines results
3/Using tools to passed the Queries to Perplexity when I asking something like where is the company, what information it hads. Meaning outside of the RAG Database and most up-to-date data. 

The table will have 8 columns:
Customer name, Item code, Description, Quantity, Unit, Price per unit, Total value, Billing date. 

This 8 columns enough data for me. From customer name, if I want any other information, Perplexity can help me. 

I choose to reduce the complexity of the data in order to 1 single client data including a very few columns but having many many rows instead, this should fit to my need, no need to get a really complex multiple relationship database. 

FYI: The description will be in Vietnamese so Local LLM need to able to handle multi-language.
"""

In [ ]:
"""
!pip install -q pandas
!pip install -q openai
!pip install -q sentence-transformers
!pip install -q llama-index-program-openai
!pip install -q llama-index-llms-openai
!pip install -q llama-index-experimental
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-embeddings-fastembed
!pip install -q llama-index-embeddings-openai
!pip install -q fastembed
!pip install -q xlrd>=2.0.1
!pip install -q openpyxl
"""

In [ ]:
import os
import ollama
import pandas as pd
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import PromptTemplate, StorageContext,load_index_from_storage, VectorStoreIndex,SimpleDirectoryReader, Document
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent
from llama_index.core.memory import ChatMemoryBuffer

In [ ]:
"""
Loading all the API from .env file
"""
dotenv_path = find_dotenv()  # Searches up the directory tree
#print(f"Loading dotenv from: {dotenv_path}")
load_dotenv(dotenv_path)
openai_api_key = os.getenv('OPENAI_API_KEY')
hf_key = os.getenv('HF_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
"""
Setuping Ollama by install Ollama, 
Then ollama pull llama3.1
"""

Ollama_MODEL = "llama3.1"  # Specifies the Ollama model to use.
Ollama_LLM = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [ ]:
"""
Setuping Deepseek (from Ollama) by install Ollama
Then ollama pull deepseek-r1:8b
"""

Deepseek_MODEL = "deepseek-r1:8b"  # Specifies the Ollama model to use.
Deepseek_LLM = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [ ]:
def load_excel_files(folder_path):
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_excel(file_path)
                all_data.append(df)
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
                # Handle the error appropriately (e.g., skip the file, raise an exception)
                return None

    if not all_data:
        print("No .xls files found in the specified folder.")
        return None

    try:
      combined_df = pd.concat(all_data, ignore_index=True)
      return combined_df
    except Exception as e:
      print(f"Error combining dataframes: {e}")
      return None

In [ ]:
# initialize empty df and load Sale_Data
df = load_excel_files("Sale_Data")
df.head()

In [ ]:
# Get the unique payer names
unique_payers = df['Company_Name'].unique()

# Display the unique payer names
#unique_payers

In [ ]:
#query_engine = PandasQueryEngine(df=df, verbose=True) #OpenAI
query_engine = PandasQueryEngine(df=df, llm=Deepseek_LLM, verbose=True)
#query_engine = PandasQueryEngine(df=df, llm=Ollama_LLM, verbose=True)

prompts = query_engine.get_prompts()

In [ ]:
new_prompt = PromptTemplate(
    """\
You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
The dataframe contains data about sale data. 
It have 8 columns: 
Company_Name is the full name of the buyer
Item_Code is the ERP 
Item_Details is the item information including the thickness, width, length of the steel
Quantity is the purchasing quantity,
Unit is the unit of purchasing quantity,
Value is the purchasing value, 
Currency is the unit of purchasing value, 
Billing Date is the date that issues the billing or invoice					

This is the result of `print(df.head())`: {df_str}

Follow these instructions: {instruction_str}
Query: {query_str}

Return the answer from the dataframe with a natural language explanation of the answer.
"Generate *only* Pandas code to answer the question. Do not include any surrounding text or keywords, such as 'python'."
Expression: """
)

query_engine.update_prompts({"pandas_prompt": new_prompt})
prompts = query_engine.get_prompts()
#print(prompts["pandas_prompt"].template)

In [ ]:
# Replace predict with create
def _predict(prompt, **kwargs):
    """Helper function to make predict calls using the Ollama API."""
    global Ollama_MODEL  # Access the global variable
    response = Ollama_LLM.chat.completions.create(
        model=Ollama_MODEL,
        messages=[
            {"role": "system", "content": "You are an pandas dataframe expert, user will gave you an query, you will respond with Python executable query code using Pandas. Follow user instruction step by step."},
            {"role": "user", "content": prompt.format(**kwargs)}
        ]
    )
    #print(prompts["pandas_prompt"].template)
    #print(prompt.format(**kwargs))
    return response.choices[0].message.content

# Monkey patch the predict method onto the Ollama_LLM object
Ollama_LLM.predict = _predict

In [ ]:
# Replace predict with create
def _predict(prompt, **kwargs):
    """Helper function to make predict calls using the Ollama API."""
    global Deepseek_MODEL  # Access the global variable
    response = Deepseek_LLM.chat.completions.create(
        model=Deepseek_MODEL,
        messages=[
            {"role": "system", "content": "You are an pandas dataframe expert, user will gave you an query, you will respond with Python executable query code using Pandas. Follow user instruction step by step."},
            {"role": "user", "content": prompt.format(**kwargs)}
        ]
    )
    print(prompts["pandas_prompt"].template)
    #print(prompt.format(**kwargs))
    return response.choices[0].message.content

# Monkey patch the predict method onto the Ollama_LLM object
Deepseek_LLM.predict = _predict

In [ ]:
"""
Since Local LLM is not smart, so I made a loop to running LLM multiple times with same prompts
until it got the answer.
"""

def check_Query_output(prompt):
    Error_check = False
    Pandas_output = None  # Initialize Pandas_output
    attempt_count = 0  # Initialize attempt counter

    while not Error_check and attempt_count < 5:  # Corrected while loop condition
        Query_Answer = query_engine.query(prompt)
        print(Query_Answer)
        if Query_Answer.metadata and 'raw_pandas_output' in Query_Answer.metadata:
            Pandas_intrusction = Query_Answer.metadata['pandas_instruction_str']
            Pandas_output = Query_Answer.metadata['raw_pandas_output']
            #print(Pandas_intrusction)
            if "error" in Pandas_output.lower():
                prompt = prompt + " " + Pandas_output + " " + Pandas_intrusction
                Error_check = False
                attempt_count += 1  # Increment attempt counter
            else:
                Error_check = True
        else:
            Pandas_output = str(Query_Answer)  # Return the string of the query answer.
            Error_check = True # exit the loop.

    if attempt_count >= 5 and "error" in Pandas_output.lower():
        Pandas_output = None  # Return empty Pandas_output after 5 attempts

    return Pandas_output

In [ ]:
'''
output = check_Query_output("What item details, quantiy and value did CÔNG TY TNHH ASABA VIET NAM MANUFACTURING company bought?")
print(output)
'''

In [ ]:
# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader, Document

# Read documents from the specified directory and load a specific document, "report.pdf".
documents = [Document(text=payer_name) for payer_name in df['Company_Name'].unique()]

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
embeddings = FastEmbedEmbedding()
index = VectorStoreIndex.from_documents(documents, embeddings=embeddings)
                                    
# Create a storage context
storage_context = StorageContext.from_defaults(vector_store=index.vector_store)

# Persist the storage context to disk
storage_context.persist()

# Convert the VectorStoreIndex object into a query engine. This query engine can be used to
# perform semantic searches on the index, matching natural language queries to the most relevant
# documents in the index.
query_engine_name = index.as_query_engine()

# Use the query engine to search for documents that are relevant to the query
# from the indexed documents based on the semantic understanding of the query.
#response = query_engine_name.query("What is company asaba full name?")

# Print the response obtained from the query. This will display the result of the semantic search,
# showing the information or documents that best match the query about the 2024 outlook.
#print(response)

In [ ]:
def get_company_name(query):
    response = Ollama_LLM.chat.completions.create(
        model=Ollama_MODEL,
        messages=[{"role": "user", "content": f"Extract the company name from this query: '{query}'. Return only the company name."}]
    )
    return response.choices[0].message.content
#get_company_name("What did company asaba buy?")

In [ ]:
get_company_name("What did company asaba buy?")

In [ ]:
def update_company_name(query_text, company, response):
    updated_query = query_text.lower().replace(company.lower(), response)
    return updated_query

In [ ]:
def update_query_with_company_name(query_text):
    
    #print(f"Original Query: {query_text}")
    company = get_company_name(query_text)
    #print(f"Extracted Company: {company}")

    response = query_engine_name.query(f"What is the company:'{company}' full name?")
    #print(f"Search Response: {response}")

    response_text = str(response)

    updated_query = update_company_name(query_text, company, response_text)
    #print(f"Updated Query: {updated_query}")

    return updated_query

In [ ]:
def process_query(message):
    updated_query = update_query_with_company_name(message)
    result = check_Query_output(updated_query)
    return result

In [ ]:
#check_Query_output(" Which company purchased the most SPCC 0.4 XẢ BĂNG by quantity (in KG)? ")

In [ ]:
process_query_tool = FunctionTool.from_defaults(
    fn=process_query, 
    description="Useful for getting information from the database"
)

In [ ]:
get_full_name_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine_name,
    description="Useful for getting full name of company from the database",
)

In [ ]:
from llama_index.llms.openai import OpenAI
llm = OpenAI()

# Manually provide context_window
context_window_size = 4000  # Replace with the actual context window size of your Ollama model
token_limit = int(context_window_size * 0.8) #Adjust the ratio as needed.

agent = ReActAgent.from_tools(
    tools=[process_query_tool, get_full_name_tool],
    llm=llm,
    verbose=True,
    memory_cls=ChatMemoryBuffer,
    memory_kwargs={"token_limit": token_limit}
)

In [ ]:
def agent_chat(message, history):
    response = agent.chat(message)
    return response.response

In [ ]:
# Create the chat interface for the agent
iface_agent = gr.ChatInterface(
    fn=agent_chat,
    chatbot=gr.Chatbot(height=300, type="messages"),
    textbox=gr.Textbox(placeholder="Enter your query here..."),
    title="LlamaIndex Agent Chat",
    description="Interact with the ReAct agent.",
    type="messages"
)

# Launch the agent interface
iface_agent.launch()